In [1]:
%autosave 0

Autosave disabled


In [2]:
import joblib
import pandas as pd
import numpy as np
import lightgbm as lgb

In [3]:
pd.set_option("display.max_rows", 18)

In [4]:
# 32日分のデータでテストする
TEST_DAYS = 32

In [5]:
#xtrain, ytrain, harai = joblib.load("../lgbm_today_20240401/20221228.pkl")

In [6]:
#ytrain = xtrain["jyuni"].copy()

In [7]:
#xtrain = xtrain.iloc[:, :226].drop(["ijyo", "jyuni"], axis=1).copy()

In [8]:
#ytrain

In [9]:
#joblib.dump((xtrain, ytrain, harai), "data.pkl")

# Read Data

In [5]:
xall, yall, harai = joblib.load("./data.pkl")

In [6]:
# 説明変数
# ある程度前処理済 詳しくはJV-Data490.xlsxのコード表を参照
# _1は1走前のデータ、_5は5走前のデータを表す
xall

,raceid,horseid,racedate,futan,umaban,wakuban,blinker,age,bataijyu,zogen,...,grade_5,wintime_5,lap_s3_5,lap_s4_5,lap_l3_5,lap_l4_5,f,ff,m,mf
0,2020010506010101,2017103291,20200105,54.0,1,1,0,3.0,432.0,4.0,...,None,NaN,NaN,NaN,NaN,NaN,1.120002e+09,1.120002e+09,1.220057e+09,1.120002e+09
1,2020010506010101,2017101861,20200105,54.0,2,1,0,3.0,424.0,-4.0,...,None,NaN,NaN,NaN,NaN,NaN,1.120002e+09,1.140004e+09,1.220063e+09,1.120002e+09
2,2020010506010101,2017103186,20200105,54.0,3,2,0,3.0,458.0,0.0,...,None,NaN,NaN,NaN,NaN,NaN,1.120002e+09,1.120002e+09,1.220061e+09,1.140006e+09
3,2020010506010101,2017102095,20200105,53.0,4,2,0,3.0,464.0,10.0,...,None,NaN,NaN,NaN,NaN,NaN,1.120002e+09,1.120002e+09,1.220062e+09,1.120002e+09
4,2020010506010101,2017103287,20200105,52.0,5,3,1,3.0,460.0,-2.0,...,None,NaN,NaN,NaN,NaN,NaN,1.120002e+09,1.140006e+09,1.220062e+09,1.140005e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143318,2022122809060912,2018105223,20221228,53.0,12,6,0,4.0,452.0,0.0,...,E,69.4,33.6,45.3,35.8,46.9,1.140007e+09,1.140005e+09,1.220065e+09,1.140006e+09
143319,2022122809060912,2017106137,20221228,56.0,13,7,0,5.0,492.0,6.0,...,E,67.3,32.8,44.1,34.5,45.4,1.120002e+09,1.140004e+09,1.220050e+09,1.120002e+09
143320,2022122809060912,2017100461,20221228,56.0,14,7,0,5.0,504.0,4.0,...,E,80.4,35.6,47.1,33.3,44.8,1.120002e+09,1.140004e+09,1.220055e+09,1.120002e+09
143321,2022122809060912,2018106389,20221228,52.0,15,8,0,4.0,434.0,2.0,...,E,68.4,33.7,45.0,34.7,45.7,1.120002e+09,1.140005e+09,1.220063e+09,1.120002e+09


In [12]:
# fは父の血統コード
# 7桁と9桁のコードが混在しているので後日修正
xall["f"].fillna(0).astype(int)

0         1120002326
1         1120002183
2         1120002251
3         1120002295
4         1120002261
             ...    
143318    1140006549
143319    1120002238
143320    1120001906
143321    1120002293
143322    1120002352
Name: f, Length: 143323, dtype: int64

In [7]:
# 目的変数（着順）
# 競争中止の場合等に0になる
yall

0          2
1         12
2          3
3         11
4          8
          ..
143318    12
143319    16
143320    11
143321     7
143322     8
Name: jyuni, Length: 143323, dtype: int64

In [8]:
# 払戻金
harai

,RaceID,TorokuTosu,SyussoTosu,FuseirituFlag,TokubaraiFlag,HenkanFlag,HenkanUma,HenkanWaku,HenkanDoWaku,PayTansyo,...,PayReserved1,PayUmatan,PaySanrenpuku,PaySanrentan,Year,MonthDay,JyoCD,Kaiji,Nichiji,RaceNum
0,2020010606010203,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '09', 'Pay': '000000200', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0913', 'Pay': '000000510', 'Ninki':...","[{'Kumi': '080913', 'Pay': '000001270', 'Ninki...","[{'Kumi': '091308', 'Pay': '000003840', 'Ninki...",2020,0106,06,01,02,03
1,2022010507010112,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '13', 'Pay': '000001530', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '1315', 'Pay': '000004070', 'Ninki':...","[{'Kumi': '061315', 'Pay': '000002720', 'Ninki...","[{'Kumi': '131506', 'Pay': '000026590', 'Ninki...",2022,0105,07,01,01,12
2,2020010506010101,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '07', 'Pay': '000000360', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0701', 'Pay': '000003650', 'Ninki':...","[{'Kumi': '010307', 'Pay': '000098210', 'Ninki...","[{'Kumi': '070103', 'Pay': '000280650', 'Ninki...",2020,0105,06,01,01,01
3,2020010606010204,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '12', 'Pay': '000000280', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '1211', 'Pay': '000003040', 'Ninki':...","[{'Kumi': '031112', 'Pay': '000000920', 'Ninki...","[{'Kumi': '121103', 'Pay': '000008340', 'Ninki...",2020,0106,06,01,02,04
4,2020010606010205,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '07', 'Pay': '000000380', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0716', 'Pay': '000001730', 'Ninki':...","[{'Kumi': '070916', 'Pay': '000002130', 'Ninki...","[{'Kumi': '071609', 'Pay': '000009690', 'Ninki...",2020,0106,06,01,02,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10363,2022122806050903,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '01', 'Pay': '000000600', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0107', 'Pay': '000004600', 'Ninki':...","[{'Kumi': '010307', 'Pay': '000023600', 'Ninki...","[{'Kumi': '010703', 'Pay': '000094870', 'Ninki...",2022,1228,06,05,09,03
10364,2022122806050902,16,16,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]","[{'Umaban': '04', 'Pay': '000000750', 'Ninki':...",...,"[{'Kumi': ' ', 'Pay': ' ', 'Ninki':...","[{'Kumi': '0406', 'Pay': '000030980', 'Ninki':...","[{'Kumi': '040611', 'Pay': '000013110', 'Ninki...","[{'Kumi': '040611', 'Pay': '000138570', 'Ninki...",2022,1228,06,05,09,02
10365,2022122809060911,13,13,"[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]

# Preprocess

In [9]:
# コードが文字列になっているので数値に変更
for col in xall.columns:
    if xall[col].dtype == "object":
        # Convert to numeric where possible
        xall[col] = pd.to_numeric(xall[col], errors='coerce')
        # Replace NaN (from non-numeric values) with 0
        xall[col] = xall[col].fillna(0)
        print(f"Converted column: {col}")

Converted column: KisyuCode
Converted column: TozaiCD
Converted column: ChokyosiCode
Converted column: BanusiCode
Converted column: course
Converted column: cls
Converted column: grade
Converted column: BreederCode
Converted column: KisyuCode_1
Converted column: course_1
Converted column: cls_1
Converted column: grade_1
Converted column: KisyuCode_2
Converted column: course_2
Converted column: cls_2
Converted column: grade_2
Converted column: KisyuCode_3
Converted column: course_3
Converted column: cls_3
Converted column: grade_3
Converted column: KisyuCode_4
Converted column: course_4
Converted column: cls_4
Converted column: grade_4
Converted column: KisyuCode_5
Converted column: course_5
Converted column: cls_5
Converted column: grade_5


In [10]:
# 2値分類にする
yall_binary = yall.apply(lambda x: 1 if x==1 else 0)

# Train Test Split

In [11]:
# racedateでデータを分割
rd = xall["racedate"].unique()[-TEST_DAYS] 

# split data
xtrain = xall[xall['racedate']<rd].reset_index(drop=True).copy()
ytrain = yall_binary[xtrain.index].reset_index(drop=True).copy()
xtest = xall[xall['racedate']>=rd].reset_index(drop=True).copy()
ytest = yall_binary[xtest.index].reset_index(drop=True).copy()

In [12]:
print(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)

(129888, 224) (129888,) (13435, 224) (13435,)


# Train

In [13]:
# parameters
num_boost_round = 100
early_stopping_round = 10
parameters = {
 'max_depth': 10,
 'min_data_in_leaf': 50,
 'learning_rate': 0.01,
 'seed': 1,
 'objective': 'binary',
 'metric': 'binary_logloss',
}

In [14]:
# validation dataを設定する（ここではtraining_data=validation_dataとしている）
xtrain_val = xtrain.copy()
ytrain_val = ytrain.copy()
xtest_val = xtrain.copy()
ytest_val = ytrain.copy()

In [15]:
# LightGBM用の設定
train_data = lgb.Dataset(xtrain_val, label=ytrain_val) 
valid_data = lgb.Dataset(xtest_val, label=ytest_val)

In [16]:
gbm = lgb.train(parameters, 
                train_data,
                valid_sets=[valid_data],
                num_boost_round=num_boost_round,
                callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_round, 
                            verbose=1)]
               )

[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=10) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1024) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=10) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=1024) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Number of positive: 9405, number of negative: 120483
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.044524 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17633
[LightGBM] [Info] Number of data points in the train set: 129888, number of used features: 212
[LightGBM] [Warning] Provided

In [17]:
# 特徴量の重要度の表示
importance = pd.DataFrame(gbm.feature_importance(), index=xtrain.columns, columns=['importance'])
#importance.sort_values("importance", ascending=False)
for k, v in importance.sort_values("importance", ascending=False).iterrows():
    print(v.name, v.importance)

odds 964
odds_5 68
honsyokin_1 45
goal_1 38
KisyuCode 38
4c_5 37
lap_s3_5 37
BreederCode 34
KisyuCode_5 31
3c_1 30
jyo_3 30
umaban_3 30
3c_5 29
KisyuCode_3 29
honsyokin_4 28
3c_4 27
time_5 26
odds_3 26
lap_s4_1 24
horseid 23
umaban 23
KisyuCode_2 22
ninki_1 22
l3_4 22
bataijyu 22
odds_1 22
honsyokin_3 21
ChokyosiCode 21
goal_3 20
wintime_4 20
4c_1 20
3c_2 20
f 20
odds_2 19
BanusiCode 19
zogen_1 19
lap_l3_2 19
l3_3 19
diff_3 19
futan_2 18
ff 18
4c_3 18
goal_5 18
lap_l3_4 17
mf 17
lap_s3_2 17
time_3 17
lap_s3_3 17
jyuni_1 17
honsyokin_2 17
diff_1 17
goal_4 16
lap_s3_1 16
jyuryo_2 16
lap_s4_5 16
nyusen_1 16
odds_4 15
diff_2 15
m 15
lap_l4_5 15
lap_l4_2 14
zogen_4 14
raceid 14
l3_1 14
l3_2 14
umaban_2 14
wintime_2 13
3c_3 13
lap_l3_1 13
umaban_1 13
lap_l4_1 12
bataijyu_2 12
wintime_3 12
lap_l4_3 12
tosu_5 12
bataijyu_5 12
futan_1 11
tosu_3 11
zogen_2 11
4c_2 11
lap_s4_3 11
diff_4 11
4c_4 11
nyusen_5 11
cls_4 11
KisyuCode_4 11
lap_l3_5 10
ninki_2 10
lap_s3_4 10
lap_l3_3 10
wakuban_4 9
winti

# Predict

In [ ]:
ypred = gbm.predict(xtest)

In [ ]:
ypred

In [ ]:
# logloss score
from sklearn.metrics import log_loss
log_loss(ytest.values, ypred)

# Evaluate

In [ ]:
def calc_mrr(ypred, ytrue):
    '''
    Args:
        ypred（numpy.array): 予想順位（1以上） [n, MAX_HORSE]
        ytrue（numpy.array): 確定順位 [n, MAX_HORSE]
    
    Returns:
        mrr (int)
    '''
    res = []
    for i in range(ytrue.shape[0]):
        #1着馬を何位と予想しているか 同着があるので小さい方を取る
        indices = np.where(ytrue[i, :] == 1) 
        if indices[0].size > 0: # 発走前のレースで1着がない場合がある
            res.append(1 / ypred[i, np.min(indices)])
    return np.mean(res)

In [ ]:
def get_reports(df_res):
    """概要

    単勝の予想結果をレポートとして出力する。

    Args:
        df_res (pd.DataFrame): get_win_resultsから出力された結果集計データ

    Returns:
        None
        
    Note:
        返還があるレースは集計対象外
    """

    #的中率
    tekichu = df_res.apply(lambda x: sum([y in x["true"] for y in x["pred"]])>=1, axis=1).astype(int).values
    print("レース数", df_res.shape[0])
    print("的中数", np.sum(tekichu))
    print("的中率", np.mean(tekichu), np.std(tekichu, ddof=1), np.std(tekichu, ddof=1) / np.sqrt(df_res.shape[0]))

    #回収率
    modoshi = df_res.apply(lambda x: sum([x["pay"][y] if y in x["true"] else 0 for y in x["pred"]]) + x["henkan"]*100, axis=1).values
    harai = df_res.apply(lambda x: len(x["pred"]), axis=1).values * 100
    print("払戻金額", np.sum(modoshi))
    print("回収率", np.mean(modoshi/harai), np.std(modoshi/harai, ddof=1), np.std(modoshi/harai, ddof=1) / np.sqrt(df_res.shape[0]))
    print("※平均・標準偏差・標準誤差")


In [ ]:
def get_win_results(df_bet, df_kekka):
    """概要

    単勝の買い目を入力し、払戻データとマージして出力する。

    Args:
        df_bet (pd.DataFrame): raceid, winを含むデータ
        df_kekka (pd.DataFrame): JRA-VANから抽出した払戻データ

    Returns:
        df_results (pd.DataFrame): レース毎に集計したデータ
        
    Note:
        同着があるのでtrueはlist、payはdictになっている
    """

    #raceidごとに集計
    res = []
    for raceid in df_bet["raceid"].tolist():
        #predict
        pred = df_bet[df_bet["raceid"]==raceid]["win"].values[0] #list
        #check win or loss
        try:
            tmp = df_kekka[df_kekka.raceid==raceid].iloc[0].to_dict() #target race
        except:
            print("払戻データに対象レースが存在しません。", raceid) 
            continue

        true = [x["Umaban"] for x in tmp["PayTansyo"] if x["Umaban"]!='  '] #同着があるのでtrueとpayはリスト
        pay = {x["Umaban"]:int(x["Pay"]) for x in tmp["PayTansyo"] if x["Umaban"]!='  '} 
        henkan = sum([int(tmp["HenkanUma"][x-1]) for x in pred])  #返還
        pred = [f"{x:02}" for x in pred]  # to str  
        res.append({"pred":pred, "true":true, "pay":pay, "henkan":henkan, "raceid":raceid})     
        
    df_res = pd.DataFrame(res)
    get_reports(df_res)

    return df_res


In [ ]:
def evaluate(xtest, ytest, ypred, harai):
      
    #jyuni レース数×18の行列にそれぞれの順位を格納する
    tmp = xtest.copy()
    tmp["jyuni"] = ytest
    tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: x["jyuni"].values)
    jyuni = np.array([[tmp[i][j] if j<len(tmp[i]) else j+1 for j in range(18)] for i in range(tmp.shape[0])]) #padding
    print("jyuni\n", jyuni)

    #pred レース数×18の行列にそれぞれの予想順位を格納する（スコアの大きい順）
    tmp = xtest.copy()
    tmp["pred"] = ypred
    tmp = tmp.groupby("raceid", as_index=False).apply(lambda x: np.argsort(np.argsort(-x["pred"].values)) + 1)
    pred = np.array([[tmp[i][j] if j<len(tmp[i]) else j+1 for j in range(18)] for i in range(tmp.shape[0])]) #padding
    print("pred\n", pred)

    #mrr
    mrr = calc_mrr(pred, jyuni)
    print("mrr\n", mrr)
        
    #df_bet
    df_bet = xtest[["raceid"]].groupby("raceid", as_index=False).apply(lambda x: x.iloc[0]) #ユニークなraceidを抽出しているだけ

    #win
    kaime = [[np.where(pred[i, :]==1)[0][0]+1] for i in range(pred.shape[0])] # スコア1位の単勝を買う #don't forget +1
    df_bet["win"] = kaime

    harai = harai.copy()
    harai.columns = [x.replace("RaceID", "raceid") for x in harai.columns]
    harai.raceid = harai.raceid.astype(int)

    #結果の表示
    df_res = get_win_results(df_bet, harai)

    return df_bet, df_res

In [ ]:
df_bet, df_res = evaluate(xtest, yall[xtest.index].reset_index(drop=True).copy(), ypred, harai)

In [ ]:
# 予想
df_bet

In [ ]:
# 予想と結果
df_res

In [ ]:
# 的中レース
df_res[df_res.pred==df_res.true]

In [ ]:
xtest["pred"] = ypred
xtest["jyuni"] = yall[xtest.index].reset_index(drop=True).copy()

In [ ]:
# 2022年12月28日 6回 阪神 9日目 12レース
xtest[xtest["raceid"]==2022122809060912]